In [1]:
import pandas as pd
import numpy as np
import cv2
import os

In [2]:
home_dir = os.getenv("HOME")
raw_data = pd.read_csv(home_dir + '/Desktop/DataPacket/data.csv')

In [3]:
raw_data.head()

,laser0,laser1,laser2,laser3,laser4,laser5,laser6,laser7,laser8,laser9,...,laser352,laser353,laser354,laser355,laser356,laser357,laser358,laser359,speed,steering_angle
0,0.227497,0.241333,0.221897,0.244679,0.220947,0.246013,0.231171,0.230307,0.242182,0.231787,...,0.226018,0.208932,0.252255,0.213605,0.241162,0.232063,0.240651,0.218328,NaN,NaN
1,0.227497,0.241333,0.221897,0.244679,0.220947,0.246013,0.231171,0.230307,0.242182,0.231787,...,0.226018,0.208932,0.252255,0.213605,0.241162,0.232063,0.240651,0.218328,NaN,NaN
2,0.258305,0.245077,0.234763,0.248366,0.213277,0.219929,0.225142,0.219998,0.241513,0.225413,...,0.235699,0.249670,0.231843,0.242016,0.245789,0.243913,0.223692,0.230089,NaN,NaN
3,0.240020,0.231830,0.231338,0.207150,0.239114,0.242090,0.231634,0.239512,0.241558,0.249161,...,0.233690,0.249073,0.239164,0.256052,0.230149,0.232954,0.233925,0.235331,NaN,NaN
4,0.250104,0.225447,0.241950,0.230710,0.236610,0.234459,0.226154,0.241565,0.260103,0.232317,...,0.244299,0.225548,0.251118,0.264531,0.228027,0.231908,0.239162,0.247615,NaN,NaN


In [5]:
#speed = raw_data["speed"].values
steering_angle = raw_data["steering_angle"].values

output = steering_angle

output[:5]

array([nan, nan, nan, nan, nan])

In [6]:
h = 240
w = 320
c = 1
img_num = len(raw_data) + 1  # Because csv lenght start with 0

img_input = []

for i in range(1, img_num):
    img = cv2.imread(home_dir + "/Desktop/DataPacket/Images/"+str(i)+".jpg", 0)
    img = cv2.resize(img, (w, h))
    
    img_input.append(img)

In [7]:
img_input = np.asarray(img_input).astype(np.float32) / 255.0

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(img_input, output, test_size=0.2, random_state=22)

del img_input
del steering_angle
del output

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout
from keras.layers import Conv2D, MaxPooling2D, Cropping2D, Lambda
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, Adam
from keras.models import Sequential
from keras.layers.core import Activation

Using TensorFlow backend.


In [10]:
if len(X_train.shape) < 4:  # We do this because when image is grey cv2 not add channel num to shape but keras want to see 1 as channel
    X_train.resize(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
    X_validation.resize(X_validation.shape[0], X_validation.shape[1], X_validation.shape[2], 1)
    
X_train.shape

(15, 240, 320, 1)

In [12]:
model = Sequential()

model.add(Conv2D(filters = 2, kernel_size = (3,3),padding = 'same',activation ='relu', input_shape = (h, w, c)))
model.add(Conv2D(filters = 4, kernel_size = (3,3),padding = 'same',activation ='relu'))
model.add(Conv2D(filters = 8, kernel_size = (5,5),padding = 'same',activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters = 16, kernel_size = (5,5),padding = 'same',activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters = 24, kernel_size = (7,7),padding = 'same',activation ='relu'))

model.add(Flatten())

'''model.add(Dense(50, activation='relu'))
model.add(Dense(28, activation='relu'))
model.add(Dense(10, activation='relu'))'''

model.add(Dense(1, activation = "sigmoid"))

model.compile(loss='mse', optimizer='sgd', metrics=['accuracy'])

In [13]:
hs = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=64, epochs=7)

Instructions for updating:
Use tf.cast instead.
Train on 15 samples, validate on 4 samples
Epoch 1/7
15/15 [==============================] - 4s 257ms/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 2/7
15/15 [==============================] - 2s 160ms/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 3/7
15/15 [==============================] - 3s 169ms/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 4/7
15/15 [==============================] - 2s 147ms/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 5/7
15/15 [==============================] - 2s 148ms/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 6/7
15/15 [==============================] - 2s 164ms/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 7/7
15/15 [==============================] - 2s 165ms/step - loss: nan - acc: 0.0000e+00 - val_loss: nan 

In [14]:
import plotly
from plotly.offline import iplot
plotly.offline.init_notebook_mode(connected=True)
import plotly.graph_objs as go

def trainingResultsGraph(model, modelCode):
    trace0 = go.Scatter(
        x = model.epoch,
        y = model.history['loss'],
        mode = 'lines',
        name = 'loss',
        line=dict(color='aquamarine')
    )

    trace1 = go.Scatter(
        x = model.epoch,
        y = model.history['val_loss'],
        mode = 'lines',
        name = 'val_loss',
        line=dict(color='darkred', dash='dash')
    )

    trace2 = go.Scatter(
        x = model.epoch,
        y = model.history['acc'],
        mode = 'lines',
        name = 'acc',
        line=dict(color='violet')
    )

    trace3 = go.Scatter(
        x = model.epoch,
        y = model.history['val_acc'],
        mode = 'lines',
        name = 'val_acc',
        line=dict(color='aqua', dash='dash')
    )

    updatemenus = list([
        dict(type="buttons",
             active=-1,
             buttons=list([
                dict(label = 'Acc Graph',
                     method = 'update',
                     args = [{'visible': [False, False, True, True]},
                             {'title': 'Trained Model'+modelCode+' training and validation accuracy'}]),
                dict(label = 'Loss Graph',
                     method = 'update',
                     args = [{'visible': [True, True, False, False]},
                             {'title': 'Trained Model'+modelCode+' training and validation loss'}]),
                dict(label = 'Both',
                     method = 'update',
                     args = [{'visible': [True, True, True, True]},
                             {'title': 'Trained Model'+modelCode+' training and validation values'}])
            ]),
        )
    ])

    data = [trace0, trace1, trace2, trace3]
    layout = dict(title='Trained Model'+modelCode+' training and validation values',
                  xaxis = dict(title = 'Epochs'),
                  updatemenus=updatemenus)

    fig = dict(data=data, layout=layout)

    iplot(fig, filename='lossGraph')
    
trainingResultsGraph(hs, "1")

In [15]:
try:
    __import__("rospkg")
    __import__("rospy")

    rospack = rospkg.RosPack()
    package_path = rospack.get_path('angelshark_deeplearning')
    
except ImportError:
    package_path = home_dir + "/Desktop"
    
    import os
    if not os.path.exists(package_path + "/model"):
        os.makedirs(package_path + "/model")
    
    print("Model saved to Desktop/model dir!!!")
    print("Please move them to deeplearning package model directory to angelshark_deeplearning/model")

Model saved to Desktop/model dir!!!
Please move them to deeplearning package model directory to angelshark_deeplearning/model


In [ ]:
import json
import matplotlib.pylab as plt
import random
from keras.callbacks import ModelCheckpoint

model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)

# Save model weights and json.
model.save_weights(package_path + '/model/model.h5')
model_json = model.to_json()
with open(package_path + '/model/model.json', 'w') as outfile:
    json.dump(model_json, outfile)

# Train and validation loss chart
print(hs.history.keys())

plt.plot(hs.history['loss'])
plt.plot(hs.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()